In [1]:
import pandas as pd

In [2]:
Invoices = pd.read_csv("../datasets/Invoices.csv", encoding='latin-1')
df = Invoices.copy()

initial_rows = df.shape[0]
# print(initial_rows)

In [3]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

df['CustomerID'] = df['CustomerID'].astype('object') 
df.loc[df['CustomerID'].notna(), 'CustomerID'] = df.loc[df['CustomerID'].notna(), 'CustomerID'].astype(int).astype(str)

df['Is_Canceled'] = df['InvoiceNo'].astype(str).str.startswith('C').astype(int)
df_cancellation = df[df['Is_Canceled'] == 1].copy()
df_sales = df[df['Is_Canceled'] == 0].copy()

df_sales.dropna(subset=['CustomerID'], inplace=True)
df_sales.dropna(subset=['Description'], inplace=True)

print(initial_rows - df_sales.shape[0] - df_cancellation.shape[0])

134697


In [4]:
SERVICE_CODES = ['POST', 'D', 'DOT', 'M', 'DOTCOM', 'CRUK', 'S', 'C2']
df_sales = df_sales[~df_sales['StockCode'].isin(SERVICE_CODES)]
print(initial_rows - df_sales.shape[0] - df_cancellation.shape[0])

136235


In [5]:
initial_sales_rows = df_sales.shape[0]
df_sales = df_sales[(df_sales['Quantity'] > 0) & (df_sales['UnitPrice'] > 0)]
print(initial_sales_rows - df_sales.shape[0])

34


In [6]:
Q_threshold = df_sales['Quantity'].quantile(0.995)
UP_threshold = df_sales['UnitPrice'].quantile(0.995)

rows_before_outlier = df_sales.shape[0]
df_sales = df_sales[(df_sales['Quantity'] <= Q_threshold) & (df_sales['UnitPrice'] <= UP_threshold)]
print(rows_before_outlier - df_sales.shape[0])

3680


In [7]:
df_final = pd.concat([df_sales, df_cancellation], ignore_index=True)
df_final['Is_UK'] = (df_final['Country'] == 'United Kingdom').astype(int)

stock_description_count = df_final.groupby('StockCode')['Description'].nunique()
inconsistent_codes = stock_description_count[stock_description_count > 1].index.tolist()
print(len(inconsistent_codes))
print(inconsistent_codes)

if inconsistent_codes:
    description_mapping = df_final[df_final['StockCode'].isin(inconsistent_codes)].groupby('StockCode')['Description'].apply(
        lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]
    )
    
    df_final.loc[df_final['StockCode'].isin(inconsistent_codes), 'Description'] = df_final['StockCode'].map(description_mapping)

    stock_description_count_after = df_final.groupby('StockCode')['Description'].nunique()
    remaining_inconsistent = stock_description_count_after[stock_description_count_after > 1].index.tolist()

    print(remaining_inconsistent)

210
['16156L', '17107D', '20622', '20725', '21109', '21112', '21175', '21232', '21243', '21507', '21811', '21818', '21899', '21928', '22129', '22134', '22135', '22179', '22197', '22199', '22246', '22268', '22285', '22286', '22287', '22383', '22407', '22416', '22466', '22584', '22595', '22597', '22602', '22632', '22776', '22777', '22778', '22785', '22804', '22812', '22813', '22837', '22847', '22849', '22896', '22900', '22932', '22937', '22939', '22949', '22950', '22952', '22953', '22963', '22965', '22972', '22985', '22986', '22999', '23015', '23020', '23028', '23029', '23031', '23032', '23035', '23040', '23041', '23043', '23044', '23045', '23047', '23056', '23057', '23061', '23065', '23066', '23068', '23071', '23075', '23079', '23081', '23086', '23091', '23103', '23104', '23106', '23107', '23109', '23126', '23127', '23128', '23130', '23131', '23145', '23148', '23168', '23169', '23188', '23191', '23194', '23196', '23197', '23203', '23205', '23209', '23212', '23214', '23229', '23231', '23

In [8]:
final_rows = df_final.shape[0]
total_dropped = initial_rows - final_rows

df_final.to_csv("../datasets/Invoices_Std.csv", index=False)